# Craft your style with AI: Building a Virtual Sylist Agent Using Amazon Bedrock and LangGraph


## Lab 2 - Creating and testing our fashion agent


In this lab, we will create create and test our Fashion Agent

![agent_architecture](../images/agent_architecture2.png)

The agent that we are creating has the capability of performing the following tasks:
- **Image-to-Image or Text-to-Image Search**: Allows users to search for products from the catalog that are similar to styles they like.
- **Text-to-Image Generation**: If the desired style is not available in the database, it can generate customized images based on the user's query.
- **Weather API Integration**: By fetching weather information from the location mentioned in the user's prompt, the agent can suggest appropriate outfits for the occasion.
- **Outpainting**: Users can upload an image and request to change the background, allowing them to visualize their preferred styles in different settings.
- **Inpainting**: Enables users to modify specific clothing items in an uploaded image, such as changing the design or color.
- **Handle human input**: the agent can require human input from the agent's actions or thoughts

### Environment setup 
This has been tested in `conda_python3` Jupyter Notebook kernel with `ml.t3.medium`

### Install the requirements
Before getting started, let's install some pre-requisite packages so that we can work LangChain and LangGraph

In [ ]:
!pip install --quiet langgraph "langchain>=0.2.13,<0.3.0" langchain-aws opensearch-py
!pip install --upgrade pydantic --quiet

### Load parameters for S3 setup
Next we will load some parameters from AOSS. Since we have already created the collection for you, you can access the required parameters using [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)

In [ ]:
import os
import boto3
from random import randint

ssm_client = boto3.client('ssm')

response = ssm_client.get_parameters(
    Names=[
        'AOSSCollectionName', 'AOSSEmbeddingSize', 'AOSSHost', 'AOSSIndexName', 'S3BucketName'
    ]
)
param_dict = {}
for parameter in response['Parameters']:
    param_dict[parameter['Name']] = parameter['Value']

os.environ["region"] = param_dict['AOSSHost'].split(".")[1]
os.environ["index_name"] = param_dict['AOSSIndexName']
os.environ["collection_name"] = param_dict['AOSSCollectionName']
os.environ["aoss_host"] = param_dict['AOSSHost']
os.environ["s3_bucket"] = param_dict['S3BucketName']
os.environ["embeddingSize"] = param_dict['AOSSEmbeddingSize']

### Importing libraries and setting up boto3 client and other dependencies
Let's import the packages for langchain and other dependencies

In [ ]:
import io
from random import randint
from helpers import download_from_s3
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage, AIMessage

from graph import graph

In [ ]:
RECURSION_LIMIT = 10
s3_client = boto3.client('s3')


def get_langgraph_config():
    thread_id = randint(0, 9999)
    langgraph_config = {
        "recursion_limit": RECURSION_LIMIT,
        "configurable": {
            "thread_id": thread_id
        }
    }
    return langgraph_config

### Looking at our agent graph
We already created the structured of our agent using the `graph.py` python file. Go ahead and check the file to learn how to create an agent using LangGraph.

Let's now take a look a the agent graph

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

### Create support function to handle user prompts

We will now create a support ticket to handle the user messages and handle image inputs.
We will also use this function to set our agent instruction using system prompt.

In [ ]:
def create_prompts(user_input, image=None, messages=None):
    system_prompt = """
        You are a smart and quirky AI Stylist.
        You answer questions about clothes to wear. 
        You have to understand the user question and give precise answer to the user.
        <Instructions> 
            1/ Try to take weather and occasion into your suggestions.
            2/ If you do not find any relevant image in the database, generate an image using your tools. 
            3/ Try to respond the user with a relevant image.
    """
    
    if image:
        if 's3://' not in image:
            # copy image into s3
            s3_client.upload_file(image, os.environ["s3_bucket"], image)
            user_input += f'The photo is avalailable at s3://{os.environ["s3_bucket"]}/{image}'
        else:
            user_input += f'The photo is avalailable at {image}'
    if messages is None:
        messages = [HumanMessage(content=user_input)]
    else:
        messages.append(HumanMessage(content=user_input))
    prompt = [SystemMessage(content=system_prompt)] + messages 
    return prompt

### Define User Question

Let's now test our agent with a user query and a photo. To do so, let's explore the inpainting functionality and make our top green.

For doing so let's use this reference image:

In [ ]:
from IPython.display import display, Image
display(Image(filename="Fashion-Dataset-Images-Western-Dress/WesternDress_Images/1001.jpg"))

We can use the prompt `Can you change the color of the top to green?` to change the top from this photo to green

In [ ]:
USER_QUERY = "Can you change the color of the top to green? "
photo = "Fashion-Dataset-Images-Western-Dress/WesternDress_Images/1001.jpg"
input_message = create_prompts(USER_QUERY, image=photo)
input_message

### Invoking the agent with the messages
Now that we have created the system prompt and user input required to invoke the agent, let's do so using LangGraph [stream](https://langchain-ai.github.io/langgraph/reference/graphs/#langgraph.graph.graph.CompiledGraph.stream) functionality.

To do so, we will create the support function `invoke_agent` that help us invoking the LangGraph agent with the user input messages

In [ ]:
def invoke_agent(input_message):
    s3_path = ""
    for out in graph.stream(
        {"messages": input_message},
        config=get_langgraph_config(),
        stream_mode="values"
    ):
        msg = next(iter(out.values()))
        last_msg = msg[-1]
        if isinstance(last_msg, ToolMessage):
            if last_msg.content.startswith("s3"):
                s3_path = last_msg.content
                print(f"\nToolMessage: S3 Path for output image: {last_msg.content}")
            else:
                print(f"\nToolMessage: {last_msg.content}")
        elif isinstance(last_msg, AIMessage):
            if isinstance(last_msg.content,list) and last_msg.content[0]["type"]=="tool_use":
                print(f"\nCalling a tool.. {last_msg.content[0]['name']} with input {last_msg.content[0]['input']}")
            print("\nAIMessage:")
            print(last_msg.content)
    s3_key = s3_path.replace("s3://"+os.environ["s3_bucket"]+"/", '')
    return s3_key

In [ ]:
s3_key = invoke_agent(input_message)

### Visualizing results
Our agent creates images and store them to [Amazon S3](https://aws.amazon.com/s3/). Let's check the result by downloading the image produced

In [ ]:
download_from_s3(os.environ["s3_bucket"], key=s3_key)

### Testing other tools

Let's now test the agent with different prompts. We will use the following prompts:
* Can you find me an image with dotted shirt and dark blue jeans?
* Generate a pink dress with white dots
* I'm going to attend a wedding at Los Angeles, can you generate a dress for me?

To test the agent `image_lookup`, `get_image_gen` and `get_weather` tools

In [ ]:
USER_QUERY = "Can you find me an image with dotted shirt and dark blue jeans? "
input_message = create_prompts(USER_QUERY)
s3_key = invoke_agent(input_message)
download_from_s3(os.environ["s3_bucket"], key=s3_key)

In [ ]:
USER_QUERY = "Generate a pink dress with white dots "
input_message = create_prompts(USER_QUERY)
s3_key = invoke_agent(input_message)
download_from_s3(os.environ["s3_bucket"], key=s3_key)

In [ ]:
USER_QUERY = "I'm going to attend a wedding at Los Angeles, can you generate a dress for me? "
input_message = create_prompts(USER_QUERY)
s3_key = invoke_agent(input_message)
download_from_s3(os.environ["s3_bucket"], key=s3_key)